<a href="https://colab.research.google.com/github/saba76/pipelines-java/blob/master/pinesacone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Pinecone
!pip install -qU pinecone-client
from pinecone import Pinecone
!pip install PyPDF2 sentence-transformers pinecone-client
!pip install -U sentence-transformers
import PyPDF2
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModel
# Initialize connection (get API key at app.pinecone.io)
api_key = "fdf2ea8b-b322-4beb-9f8b-835f21d1b1e4"
pc = Pinecone(api_key=api_key)



Initialization of pinecone, BERT , PDF etc

In [ ]:
pc.list_indexes()

verifying the indexes in Pinecone

In [131]:
def extract_text_from_pdf_and_split_into_pages(file_path):
    pages_text = []
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page_text = reader.pages[page_number].extract_text()
            pages_text.append(page_text)
    return pages_text


Text Extraction from PDF method definition

In [ ]:
drive_path='/content/drive/MyDrive/panna/'
file_path = drive_path + 'symptomsAutistic.pdf'
pages=extract_text_from_pdf_and_split_into_pages(file_path)


Source file path from google drive

Later text proprocessing - insert code here

In [133]:
import numpy as np
model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings_list=[]
content_list=[]
for page_text in pages:
  page_embedding = model.encode(page_text)
  embeddings_list.append(page_embedding.tolist())
  content_list.append(page_text)
  embeddings_array=np.array(embeddings_list)


In [134]:
num_embeddings = len(embeddings_array)
embedding_shape = embeddings_array[0].shape
print("Number of embeddings:", num_embeddings)
print("Shape of embeddings:", embedding_shape)

Number of embeddings: 113
Shape of embeddings: (768,)


In [ ]:
index = pc.Index("bert-saba-language")
for i, embedding in enumerate(embeddings_array):
    vector = {"main": f"vector_{i}", "values": embedding.tolist(),"metadata":{"splitting_chunks": f"split_{i}","content":content_list[i]}}
    index.upsert(vectors=[{"id": vector["main"], "values": vector["values"], "metadata": vector["metadata"]}])


In [ ]:

model = SentenceTransformer('sentence-transformers/LaBSE')
index = pc.Index("bert-saba-language")
query_text = "this sensory signal is often misunderstood"
vector = model.encode([query_text])[0].tolist() # convert to list
results = index.query(vector=vector, top_k=1, include_metadata=True)
print(results)
